# Imports & Loading Dataset

In [ ]:
import numpy as np
import pandas as pd

## Requirements

In [ ]:
%pip install transformers evaluate langchain datasets

## Upload files to collab

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive/')
# data = files.upload()
data = pd.read_json('/content/drive/My Drive/dataset.json')

# Preprocessing

In [ ]:
data.head()

In [ ]:
print(data.describe())
print(data.info())

In [ ]:
def preprocessing(df):
  df['review_text'] = df['review_text'].str.lower()

In [ ]:
preprocessing(data)
data.head()

In [ ]:
data.info()

# Zero-shot Shot

In [ ]:
#from langchain_huggingface.llms import HuggingFacePipeline

import json
import random
from transformers import pipeline

def rate_reviews(data, n_blocks=8, block_size=10000, sample_size=100, output_file='review_ratings.json'):
    # Carrega o modelo zero-shot
    pipe = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")

    # Define as labels de classificação
    candidate_labels = ["0 stars", "1 stars", "2 stars", "3 stars", "4 stars", "5 stars"]

    # Lista para armazenar os resultados
    results = []

    # Itera sobre cada bloco de 10.000 entradas
    for block_num in range(n_blocks):
        start_idx = block_num * block_size
        end_idx = min((block_num + 1) * block_size, len(data['review_text']))

        # Seleciona 100 reviews aleatoriamente do bloco atual
        if end_idx > start_idx:
            block_reviews = data['review_text'][start_idx:end_idx]
            selected_reviews = block_reviews[:sample_size]  # Pegamos as 100 primeiras reviews do bloco

            # Processa as 100 reviews selecionadas
            for review_text in selected_reviews:
                # Cria o prompt para classificação
                prompt = "Your objective is to read user reviews for books and determine the final rating given by the user on a scale of 0 to 5 stars. Being 0 the lowest score and 5 the highest."
                output = pipe(prompt + review_text, candidate_labels)

                # Armazena o resultado (rótulo de maior pontuação) e a review na lista
                results.append({
                    "review": review_text,
                    "rating": output['labels'][0]
                })

    # Salva os resultados em um arquivo JSON
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4)

# Executa a função, processando 100 reviews de cada bloco de 10.000
rate_reviews(data, n_blocks=8, block_size=10000, sample_size=1, output_file='review_ratings.json')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Few Shot

In [ ]:

from transformers import pipeline

model_id = "facebook/bart-large-mnli"
pipe = pipeline("zero-shot-classification", model=model_id)

candidate_labels =["0 Stars", "1 Stars", "2 Stars", "3 Stars", "4 Stars", "5 Stars"]

examples_text = [
    data['review_text'][17],
    data['review_text'][18],
    data['review_text'][19],
  ]

examples_ratings = [
    data['rating'][17],
    data['rating'][18],
    data['rating'][19],
]

query = data['review_text'][10]
label = data['rating'][10]

prompt = """Your objective is to read user reviews for books and determine the final rating given by the user on a scale of 1 to 5 stars. Being 0 the lowest score and 5 the highest. First you'll get 3 examples of
Text: {review text}
Rating: {rating on a scale of 0 to 5}

Examples:
"""

fo = prompt + "\n Text:" + examples_text[0] + "\n Rating:" + str(examples_ratings[0]) + "\n" + " Text:" + examples_text[1] + "\n Rating:" + str(examples_ratings[1]) + "\n" + " Text:" + examples_text[2] + "\n Rating:" + str(examples_ratings[2]) + "\n" + " Text:" + query



output2 = pipe(fo, candidate_labels)


print(output2)


In [ ]:
print(query)
print(label)
data.iloc[10:11]


# Fine-tuning Approach

## Sample Dataset

In [ ]:
# Sample & Split Dataset
print(len(data)) # 80k

data = data[['review_text', 'rating']]


sample = data.sample(frac=0.0625) # gives us 5k samples
sample.rename(columns={"review_text": 'text'}, inplace=True)
sample.head()

## Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(sample['text'], sample['rating'], test_size=0.3)

from datasets import Dataset

def to_dataset(x_train, x_test, y_train, y_test):
    train_set = Dataset.from_dict({
            'text': x_train,
            'label': y_train
    })

    test_set = Dataset.from_dict({
            'text': x_test,
            'label': y_test
    })
    return train_set, test_set

train_set, test_set = to_dataset(x_train, x_test, y_train, y_test)

## Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

token_train_data = train_set.map(preprocess_function, batched=True)

token_test_data = test_set.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(

    "distilbert/distilbert-base-uncased", num_labels=6
)

print(model)

### Prepare evaluation

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
training_args = TrainingArguments(

    output_dir="my_awesome_model",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=2,

    weight_decay=0.01,

    eval_strategy="epoch",

    save_strategy="epoch",

    load_best_model_at_end=True

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=token_train_data,

    eval_dataset=token_test_data,

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

)

trainer.train()
# Fuck this actually worked..